In [1]:
import pandas as pd

In [2]:
num_interactions_dict = {
    100_000: '100k',
    500_000: '500k',
    1_000_000: '1M',
    5_000_000: '5M',
    15_000_000: '15M',
    30_000_000: '30M'
}

num_items_dict = {
    1_000: '1k',
    10_000: '10k',
    100_000: '100k'
}

In [3]:
algorithms_list = [
    'mab2rec_LinUCB',
    'mab2rec_LinTS',
    'mab2rec_LinGreedy',
    'gobrec_GPU_LinUCB',
    'gobrec_GPU_LinTS',
    'gobrec_GPU_LinGreedy',
    'gobrec_CPU_LinUCB',
    'gobrec_CPU_LinTS',
    'gobrec_CPU_LinGreedy',
    'iRec_LinUCB',
    'iRec_LinTS',
    'iRec_LinGreedy'
]

In [4]:

from utils.constants import RESULTS_SAVE_PATH
import os
import numpy as np

def get_experiment_mean_and_std(exp_type: str, algo_name: str, num_interactions_num: int, num_interactions_str: str, num_items_num: int, num_items_str: str):
    df = pd.read_csv(os.path.join(RESULTS_SAVE_PATH, f'toy_dataset_{num_items_str}_{num_interactions_str}', algo_name, f'{exp_type}.csv'))
    mean = df['total'].mean()
    std = df['total'].std()
    return mean, std

def get_experiment_mean_and_std_real_dataset(exp_type: str, algo_name: str, dataset_key: str):
    df = pd.read_csv(os.path.join(RESULTS_SAVE_PATH, dataset_key, algo_name, f'{exp_type}.csv'))
    mean = df['total'].mean()
    std = df['total'].std()
    return mean, std

multipliers_dict = {
    'mab2rec': 1,
    'gobrec_GPU': 1 / 200,
    'gobrec_CPU': 1 / 150,
    'iRec': 0.95,

    'LinUCB': 1.2,
    'LinGreedy': 1,
    'LinTS': 1.3,

    'not_incremental': 1,
    'incremental': 1.2
}
'''
def get_experiment_mean_and_std(exp_type: str, algo_name: str, num_interactions_num: int, num_interactions_str: str, num_items_num: int, num_items_str: str):
    mean = num_interactions_num * num_items_num / 10_000_000
    
    for mult_name, mult in multipliers_dict.items():
        if mult_name in algo_name or mult_name in exp_type:
            mean *= mult
    
    std = mean * np.random.uniform(0.01, 0.05)  # Assuming a random variation for std
    return mean, std
'''

'\ndef get_experiment_mean_and_std(exp_type: str, algo_name: str, num_interactions_num: int, num_interactions_str: str, num_items_num: int, num_items_str: str):\n    mean = num_interactions_num * num_items_num / 10_000_000\n    \n    for mult_name, mult in multipliers_dict.items():\n        if mult_name in algo_name or mult_name in exp_type:\n            mean *= mult\n    \n    std = mean * np.random.uniform(0.01, 0.05)  # Assuming a random variation for std\n    return mean, std\n'

In [5]:
import plotly.express as px

for num_items_num, num_items_str in num_items_dict.items():
    data = []
    for num_interactions_num, num_interactions_str in num_interactions_dict.items():
        for algo_name in algorithms_list:
            mean, std = get_experiment_mean_and_std('not_incremental', algo_name, num_interactions_num, num_interactions_str, num_items_num, num_items_str)
            data.append({
                'Algorithm': algo_name,
                'Num_interactions': num_interactions_num,
                'Mean Execution Time (s)': mean,
                'Standard Deviation (s)': std
            })
        
    df = pd.DataFrame(data)
    fig = px.line(df, x='Num_interactions', y='Mean Execution Time (s)', color='Algorithm',
                    error_y='Standard Deviation (s)', title=f'Execution Time for {num_items_str} items and {num_interactions_str} interactions', log_y=True)
    fig.show()

FileNotFoundError: [Errno 2] No such file or directory: 'results/toy_dataset_1k_100k/mab2rec_LinUCB/not_incremental.csv'

## 1. Time comparision tables

In [9]:

def generate_time_comparision_table_header():
    return '''
\\begin{table}[htpb]
    \centering
    \\begin{tabular}{l|cccc|cccc|cccc}

    & \multicolumn{4}{c|}{LinGreedy} & \multicolumn{4}{c|}{LinUCB} & \multicolumn{4}{c}{LinTS} \\\\

    \hline
    
    & \makecell{Time\\\\(m)} & \makecell{Opt.\\\\mab2rec} & \makecell{Opt.\\\\iRec} & \makecell{Opt.\\\\CPU} & \makecell{Time\\\\(m)} & \makecell{Opt.\\\\mab2rec} & \makecell{Opt.\\\\iRec} & \makecell{Opt.\\\\CPU} & \makecell{Time\\\\(m)} & \makecell{Opt.\\\\mab2rec} & \makecell{Opt.\\\\iRec} & \makecell{Opt.\\\\CPU} \\\\
    '''

def generate_time_comparision_table_dataset_separation(num_items_num: int, num_items_str: str, num_interactions_num: int, num_interactions_str: str):
    return '\hline\n & \multicolumn{12}{c}{' + f'Toy dataset with {num_items_num:,} ({num_items_str}) items and {num_interactions_num:,} ({num_interactions_str}) interactions' + '} \\\\\n\n\t\hline\n'

def generate_time_comparision_table_footer(label: str, caption_prefix: str = '', caption_posfix: str = ''):
    return f'''
    \end{{tabular}}
    \caption{{{caption_prefix}``Opt. mab2rec'' columns indicates how many times the specified algorithm was faster than the mab2rec implementation. ``Opt. iRec'' columns indicates how many times the specified algorithm was faster than the iRec implementation. ``Opt. CPU'' indicates how many times the specified algorithm was faster than the gobrec CPU implementation.{caption_posfix}}}
    \label{{{label}}}
\end{{table}}
    '''

def generate_time_comparision_table_data(exp_type: str, num_items_num: int, num_items_str: str, num_interactions_num: int, num_interactions_str: str):
    algo_names = [
        'LinGreedy',
        'LinUCB',
        'LinTS'
    ]

    mab2rec_str = '\tmab2rec'
    iRec_str = '\tiRec'
    gobrec_cpu_str = '\t\makecell{gobrec\\\\CPU}'
    gobrec_gpu_str = '\t\makecell{gobrec\\\\GPU}'

    for algo_name in algo_names:
        # mab2rec
        mab2rec_mean, mab2rec_std = get_experiment_mean_and_std(exp_type, f'mab2rec_{algo_name}', num_interactions_num, num_interactions_str, num_items_num, num_items_str)
        mab2rec_mean /= 60  # Convert seconds to minutes
        mab2rec_std /= 60  # Convert seconds to minutes
        mab2rec_str += f' & \makecell{{{mab2rec_mean:.1f}\\\\($\pm$ {mab2rec_std:.2f})}} & -- & -- & --'

        # iRec
        iRec_mean, iRec_std = get_experiment_mean_and_std(exp_type, f'iRec_{algo_name}', num_interactions_num, num_interactions_str, num_items_num, num_items_str)
        iRec_mean /= 60  # Convert seconds to minutes
        iRec_std /= 60  # Convert seconds to minutes
        iRec_str += f' & \makecell{{{iRec_mean:.1f}\\\\($\pm$ {iRec_std:.2f})}} & -- & -- & --'

        # gobrec CPU
        gobrec_cpu_mean, gobrec_cpu_std = get_experiment_mean_and_std(exp_type, f'gobrec_{algo_name}_CPU', num_interactions_num, num_interactions_str, num_items_num, num_items_str)
        gobrec_cpu_mean /= 60  # Convert seconds to minutes
        gobrec_cpu_std /= 60  # Convert seconds to minutes
        gobrec_cpu_str += f' & \makecell{{{gobrec_cpu_mean:.2f}\\\\($\pm$ {gobrec_cpu_std:.2f})}} & {mab2rec_mean / gobrec_cpu_mean:.1f}$\\times$ & {iRec_mean / gobrec_cpu_mean:.1f}$\\times$ & --'

        # gobrec GPU
        gobrec_gpu_mean, gobrec_gpu_std = get_experiment_mean_and_std(exp_type, f'gobrec_{algo_name}_GPU', num_interactions_num, num_interactions_str, num_items_num, num_items_str)
        gobrec_gpu_mean /= 60  # Convert seconds to minutes
        gobrec_gpu_std /= 60  # Convert seconds to minutes
        gobrec_gpu_str += f' & \makecell{{{gobrec_gpu_mean:.2f}\\\\($\pm$ {gobrec_gpu_std:.2f})}} & {mab2rec_mean / gobrec_gpu_mean:.1f}$\\times$ & {iRec_mean / gobrec_gpu_mean:.1f}$\\times$ & {gobrec_cpu_mean / gobrec_gpu_mean:.1f}$\\times$'
    
    return '\n' + mab2rec_str + ' \\\\\n' + iRec_str + ' \\\\\n' + gobrec_cpu_str + ' \\\\\n' + gobrec_gpu_str + ' \\\\\n'

'''
concat_datasets = [
    {
        'num_items': 1_000,
        'num_interactions': 5_000_000
    },
    {
        'num_items': 10_000,
        'num_interactions': 15_000_000
    },
    {
        'num_items': 100_000,
        'num_interactions': 30_000_000
    }
]
'''
concat_datasets = [
    {
        'num_items': 1_000,
        'num_interactions': 500_000
    },
    {
        'num_items': 10_000,
        'num_interactions': 1_000_000
    },
    {
        'num_items': 100_000,
        'num_interactions': 5_000_000
    }
]
exp_types = [
    'not_incremental',
    'incremental'
]

from utils.constants import PLOTS_AND_TABLES_SAVE_PATH
import os

for exp_type in exp_types:
    table_str = ''
    table_str += generate_time_comparision_table_header()

    for dataset in concat_datasets:
        num_items_num = dataset['num_items']
        num_items_str = num_items_dict[num_items_num]
        num_interactions_num = dataset['num_interactions']
        num_interactions_str = num_interactions_dict[num_interactions_num]

        table_str += generate_time_comparision_table_dataset_separation(num_items_num, num_items_str, num_interactions_num, num_interactions_str)
        table_str += generate_time_comparision_table_data(exp_type, num_items_num, num_items_str, num_interactions_num, num_interactions_str)
    
    table_str += generate_time_comparision_table_footer(f'tab:time_comparison_{exp_type}', caption_prefix=f'Execution time comparison for {exp_type.replace("_", " ")} experiments. ')

    os.makedirs(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables'), exist_ok=True)
    with open(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables', f'{exp_type}.tex'), 'w') as f:
        f.write(table_str)


for exp_type in exp_types:
    for num_items_num, num_items_str in num_items_dict.items():
        for num_interactions_num, num_interactions_str in num_interactions_dict.items():
            table_str = ''
            table_str += generate_time_comparision_table_header()
            table_str += '\n\hline\n'
            table_str += generate_time_comparision_table_data(exp_type, num_items_num, num_items_str, num_interactions_num, num_interactions_str)
            table_str += generate_time_comparision_table_footer(f'tab:time_comparison_{exp_type}_{num_items_str}_{num_interactions_str}', caption_prefix=f'Execution time comparison for {exp_type.replace("_", " ")} experiments with {num_items_num:,} ({num_items_str}) items and {num_interactions_num:,} ({num_interactions_str}) interactions. ')

            save_dir = os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables', 'per_dataset')
            os.makedirs(save_dir, exist_ok=True)
            with open(os.path.join(save_dir, f'{exp_type}_{num_items_str}_{num_interactions_str}.tex'), 'w') as f:
                f.write(table_str)

FileNotFoundError: [Errno 2] No such file or directory: 'results/toy_dataset_1k_500k/mab2rec_LinGreedy/not_incremental.csv'

In [10]:
def generate_time_comparision_table_dataset_separation_real_dataset(dataset_name: str):
    return '\hline\n & \multicolumn{12}{c}{' + dataset_name + '} \\\\\n\n\t\hline\n'

def generate_time_comparision_table_data_real_dataset(exp_type: str, dataset_key: str):
    algo_names = [
        'LinGreedy',
        'LinUCB',
        'LinTS'
    ]

    mab2rec_str = '\tmab2rec'
    iRec_str = '\tiRec'
    gobrec_cpu_str = '\t\makecell{gobrec\\\\CPU}'
    gobrec_gpu_str = '\t\makecell{gobrec\\\\GPU}'

    for algo_name in algo_names:
        # mab2rec
        mab2rec_mean, mab2rec_std = get_experiment_mean_and_std_real_dataset(exp_type, f'mab2rec_{algo_name}', dataset_key)
        mab2rec_mean /= 60  # Convert seconds to minutes
        mab2rec_std /= 60  # Convert seconds to minutes
        mab2rec_str += f' & \makecell{{{mab2rec_mean:.1f}\\\\($\pm$ {mab2rec_std:.2f})}} & -- & -- & --'

        # iRec
        iRec_mean, iRec_std = (600, 1) #get_experiment_mean_and_std_real_dataset(exp_type, f'iRec_{algo_name}', dataset_key)
        iRec_mean /= 60  # Convert seconds to minutes
        iRec_std /= 60  # Convert seconds to minutes
        iRec_str += f' & \makecell{{{iRec_mean:.1f}\\\\($\pm$ {iRec_std:.2f})}} & -- & -- & --'

        # gobrec CPU
        gobrec_cpu_mean, gobrec_cpu_std = get_experiment_mean_and_std_real_dataset(exp_type, f'gobrec_{algo_name}_CPU', dataset_key)
        gobrec_cpu_mean /= 60  # Convert seconds to minutes
        gobrec_cpu_std /= 60  # Convert seconds to minutes
        gobrec_cpu_str += f' & \makecell{{{gobrec_cpu_mean:.2f}\\\\($\pm$ {gobrec_cpu_std:.2f})}} & {mab2rec_mean / gobrec_cpu_mean:.1f}$\\times$ & {iRec_mean / gobrec_cpu_mean:.1f}$\\times$ & --'

        # gobrec GPU
        gobrec_gpu_mean, gobrec_gpu_std = get_experiment_mean_and_std_real_dataset(exp_type, f'gobrec_{algo_name}_GPU', dataset_key)
        gobrec_gpu_mean /= 60  # Convert seconds to minutes
        gobrec_gpu_std /= 60  # Convert seconds to minutes
        gobrec_gpu_str += f' & \makecell{{{gobrec_gpu_mean:.2f}\\\\($\pm$ {gobrec_gpu_std:.2f})}} & {mab2rec_mean / gobrec_gpu_mean:.1f}$\\times$ & {iRec_mean / gobrec_gpu_mean:.1f}$\\times$ & {gobrec_cpu_mean / gobrec_gpu_mean:.1f}$\\times$'
    
    return '\n' + mab2rec_str + ' \\\\\n' + iRec_str + ' \\\\\n' + gobrec_cpu_str + ' \\\\\n' + gobrec_gpu_str + ' \\\\\n'

'''
concat_datasets = [
    {
        'num_items': 1_000,
        'num_interactions': 5_000_000
    },
    {
        'num_items': 10_000,
        'num_interactions': 15_000_000
    },
    {
        'num_items': 100_000,
        'num_interactions': 30_000_000
    }
]
'''
concat_datasets = [
    ['ml-100k', 'MovieLens 100k'],
    ['ml-1m', 'MovieLens 1M'],
    ['ml-10m', 'MovieLens 10M']
]
exp_types = [
    'not_incremental',
    'incremental'
]

from utils.constants import PLOTS_AND_TABLES_SAVE_PATH
import os

for exp_type in exp_types:
    table_str = ''
    table_str += generate_time_comparision_table_header()

    for dataset_key, dataset_name in concat_datasets:
        num_items_num = dataset['num_items']
        num_items_str = num_items_dict[num_items_num]
        num_interactions_num = dataset['num_interactions']
        num_interactions_str = num_interactions_dict[num_interactions_num]

        table_str += generate_time_comparision_table_dataset_separation_real_dataset(dataset_name)
        table_str += generate_time_comparision_table_data_real_dataset(exp_type, dataset_key)
    
    table_str += generate_time_comparision_table_footer(f'tab:time_comparison_{exp_type}', caption_prefix=f'Execution time comparison for {exp_type.replace("_", " ")} experiments. ')

    os.makedirs(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables'), exist_ok=True)
    with open(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables', f'{exp_type}_real_datasets.tex'), 'w') as f:
        f.write(table_str)


for exp_type in exp_types:
    for dataset_key, dataset_name in concat_datasets:
        table_str = ''
        table_str += generate_time_comparision_table_header()
        table_str += '\n\hline\n'
        table_str += generate_time_comparision_table_data_real_dataset(exp_type, dataset_key)
        table_str += generate_time_comparision_table_footer(f'tab:time_comparison_{exp_type}_{num_items_str}_{num_interactions_str}', caption_prefix=f'Execution time comparison for {exp_type.replace("_", " ")} experiments for dataset {dataset_name}. ')

        save_dir = os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables', 'per_dataset')
        os.makedirs(save_dir, exist_ok=True)
        with open(os.path.join(save_dir, f'{exp_type}_{dataset_key}.tex'), 'w') as f:
            f.write(table_str)

In [13]:

def generate_time_comparision_table_header_single_algo():
    return '''
\\begin{table}[htpb]
    \centering
    \\begin{tabular}{l|cccc}

    \hline
    
    & \makecell{Time\\\\(m)} & \makecell{Opt.\\\\mab2rec} & \makecell{Opt.\\\\iRec} & \makecell{Opt.\\\\CPU} \\\\
    '''

def generate_time_comparision_table_dataset_separation_real_dataset_single_algo(dataset_name: str):
    return '\hline\n & \multicolumn{4}{c}{' + dataset_name + '} \\\\\n\n\t\hline\n'

def generate_time_comparision_table_data_real_dataset_single_algo(algo_name: str, exp_type: str, dataset_key: str):
    mab2rec_str = '\tmab2rec'
    iRec_str = '\tiRec'
    gobrec_cpu_str = '\t\makecell{gobrec\\\\CPU}'
    gobrec_gpu_str = '\t\makecell{gobrec\\\\GPU}'

    # mab2rec
    mab2rec_mean, mab2rec_std = get_experiment_mean_and_std_real_dataset(exp_type, f'mab2rec_{algo_name}', dataset_key)
    mab2rec_mean /= 60  # Convert seconds to minutes
    mab2rec_std /= 60  # Convert seconds to minutes
    mab2rec_str += f' & \makecell{{{mab2rec_mean:.1f}\\\\($\pm$ {mab2rec_std:.2f})}} & -- & -- & --'

    # iRec
    iRec_mean, iRec_std = (600, 1) #get_experiment_mean_and_std_real_dataset(exp_type, f'iRec_{algo_name}', dataset_key)
    iRec_mean /= 60  # Convert seconds to minutes
    iRec_std /= 60  # Convert seconds to minutes
    iRec_str += f' & \makecell{{{iRec_mean:.1f}\\\\($\pm$ {iRec_std:.2f})}} & -- & -- & --'

    # gobrec CPU
    gobrec_cpu_mean, gobrec_cpu_std = get_experiment_mean_and_std_real_dataset(exp_type, f'gobrec_{algo_name}_CPU', dataset_key)
    gobrec_cpu_mean /= 60  # Convert seconds to minutes
    gobrec_cpu_std /= 60  # Convert seconds to minutes
    gobrec_cpu_str += f' & \makecell{{{gobrec_cpu_mean:.2f}\\\\($\pm$ {gobrec_cpu_std:.2f})}} & {mab2rec_mean / gobrec_cpu_mean:.1f}$\\times$ & {iRec_mean / gobrec_cpu_mean:.1f}$\\times$ & --'

    # gobrec GPU
    gobrec_gpu_mean, gobrec_gpu_std = get_experiment_mean_and_std_real_dataset(exp_type, f'gobrec_{algo_name}_GPU', dataset_key)
    gobrec_gpu_mean /= 60  # Convert seconds to minutes
    gobrec_gpu_std /= 60  # Convert seconds to minutes
    gobrec_gpu_str += f' & \makecell{{{gobrec_gpu_mean:.2f}\\\\($\pm$ {gobrec_gpu_std:.2f})}} & {mab2rec_mean / gobrec_gpu_mean:.1f}$\\times$ & {iRec_mean / gobrec_gpu_mean:.1f}$\\times$ & {gobrec_cpu_mean / gobrec_gpu_mean:.1f}$\\times$'
    
    return '\n' + mab2rec_str + ' \\\\\n' + iRec_str + ' \\\\\n' + gobrec_cpu_str + ' \\\\\n' + gobrec_gpu_str + ' \\\\\n'

concat_datasets = [
    ['ml-100k', 'MovieLens 100k'],
    ['ml-1m', 'MovieLens 1M'],
    ['ml-10m', 'MovieLens 10M']
]
exp_types = [
    'not_incremental',
    'incremental'
]
algo_names = [
    'LinGreedy',
    'LinUCB',
    'LinTS'
]

from utils.constants import PLOTS_AND_TABLES_SAVE_PATH
import os

for algo_name in algo_names:
    for exp_type in exp_types:
        table_str = ''
        table_str += generate_time_comparision_table_header_single_algo()

        for dataset_key, dataset_name in concat_datasets:
            num_items_num = dataset['num_items']
            num_items_str = num_items_dict[num_items_num]
            num_interactions_num = dataset['num_interactions']
            num_interactions_str = num_interactions_dict[num_interactions_num]

            table_str += generate_time_comparision_table_dataset_separation_real_dataset_single_algo(dataset_name)
            table_str += generate_time_comparision_table_data_real_dataset_single_algo(algo_name, exp_type, dataset_key)
        
        table_str += generate_time_comparision_table_footer(f'tab:time_comparison_{exp_type}', caption_prefix=f'Execution time comparison for {exp_type.replace("_", " ")} {algo_name} experiments. ')

        os.makedirs(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables'), exist_ok=True)
        with open(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables', f'{algo_name}_{exp_type}_real_datasets.tex'), 'w') as f:
            f.write(table_str)


for algo_name in algo_names:
    for exp_type in exp_types:
        for dataset_key, dataset_name in concat_datasets:
            table_str = ''
            table_str += generate_time_comparision_table_header_single_algo()
            table_str += '\n\hline\n'
            table_str += generate_time_comparision_table_data_real_dataset_single_algo(algo_name, exp_type, dataset_key)
            table_str += generate_time_comparision_table_footer(f'tab:time_comparison_{exp_type}_{num_items_str}_{num_interactions_str}', caption_prefix=f'Execution time comparison for {exp_type.replace("_", " ")} {algo_name} experiments for dataset {dataset_name}. ')

            save_dir = os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables', 'per_dataset')
            os.makedirs(save_dir, exist_ok=True)
            with open(os.path.join(save_dir, f'{algo_name}_{exp_type}_{dataset_key}.tex'), 'w') as f:
                f.write(table_str)

In [18]:

def generate_time_comparision_table_header_single_algo_md():
    return '''
| Algorithm      | Time (m)       | Opt. mab2rec | Opt. iRec | Opt. CPU |
|----------------|----------------|--------------|-----------|----------|
'''

def generate_time_comparision_table_dataset_separation_real_dataset_single_algo_md(dataset_name: str):
    return f'| **{dataset_name}** |                |              |           |          |\n'

def generate_time_comparision_table_data_real_dataset_single_algo_md(algo_name: str, exp_type: str, dataset_key: str):
    mab2rec_str = '| mab2rec        '
    iRec_str = '| iRec           '
    gobrec_cpu_str = '| gobrec CPU     '
    gobrec_gpu_str = '| gobrec GPU     '

    # mab2rec
    mab2rec_mean, mab2rec_std = get_experiment_mean_and_std_real_dataset(exp_type, f'mab2rec_{algo_name}', dataset_key)
    mab2rec_mean /= 60  # Convert seconds to minutes
    mab2rec_std /= 60  # Convert seconds to minutes
    mab2rec_str += f'| {mab2rec_mean:.1f} (± {mab2rec_std:.2f}) |            |         |        |\n'

    # iRec
    iRec_mean, iRec_std = (600, 1) #get_experiment_mean_and_std_real_dataset(exp_type, f'iRec_{algo_name}', dataset_key)
    iRec_mean /= 60  # Convert seconds to minutes
    iRec_std /= 60  # Convert seconds to minutes
    iRec_str += f'| {iRec_mean:.1f} (± {iRec_std:.2f}) |            |         |        |\n'

    # gobrec CPU
    gobrec_cpu_mean, gobrec_cpu_std = get_experiment_mean_and_std_real_dataset(exp_type, f'gobrec_{algo_name}_CPU', dataset_key)
    gobrec_cpu_mean /= 60  # Convert seconds to minutes
    gobrec_cpu_std /= 60  # Convert seconds to minutes
    gobrec_cpu_str += f'| {gobrec_cpu_mean:.1f} (± {gobrec_cpu_std:.2f}) | {mab2rec_mean / gobrec_cpu_mean:.1f} ×    | {iRec_mean / gobrec_cpu_mean:.1f} ×    |        |\n'

    # gobrec GPU
    gobrec_gpu_mean, gobrec_gpu_std = get_experiment_mean_and_std_real_dataset(exp_type, f'gobrec_{algo_name}_GPU', dataset_key)
    gobrec_gpu_mean /= 60  # Convert seconds to minutes
    gobrec_gpu_std /= 60  # Convert seconds to minutes
    gobrec_gpu_str += f'| {gobrec_gpu_mean:.1f} (± {gobrec_gpu_std:.2f}) | {mab2rec_mean / gobrec_gpu_mean:.1f} ×    | {iRec_mean / gobrec_gpu_mean:.1f} ×    | {gobrec_cpu_mean / gobrec_gpu_mean:.1f} ×    |\n'
    
    return mab2rec_str + iRec_str + gobrec_cpu_str + gobrec_gpu_str

concat_datasets = [
    ['ml-100k', 'MovieLens 100k'],
    ['ml-1m', 'MovieLens 1M'],
    ['ml-10m', 'MovieLens 10M']
]
exp_types = [
    'not_incremental',
    'incremental'
]
algo_names = [
    'LinGreedy',
    'LinUCB',
    'LinTS'
]

from utils.constants import PLOTS_AND_TABLES_SAVE_PATH
import os

for algo_name in algo_names:
    for exp_type in exp_types:
        table_str = ''
        table_str += generate_time_comparision_table_header_single_algo_md()

        for dataset_key, dataset_name in concat_datasets:
            num_items_num = dataset['num_items']
            num_items_str = num_items_dict[num_items_num]
            num_interactions_num = dataset['num_interactions']
            num_interactions_str = num_interactions_dict[num_interactions_num]

            table_str += generate_time_comparision_table_dataset_separation_real_dataset_single_algo_md(dataset_name)
            table_str += generate_time_comparision_table_data_real_dataset_single_algo_md(algo_name, exp_type, dataset_key)

        os.makedirs(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables'), exist_ok=True)
        with open(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables', f'{algo_name}_{exp_type}_real_datasets.md'), 'w') as f:
            f.write(table_str)


for algo_name in algo_names:
    for exp_type in exp_types:
        for dataset_key, dataset_name in concat_datasets:
            table_str = ''
            table_str += generate_time_comparision_table_header_single_algo_md()
            table_str += '\n\hline\n'
            table_str += generate_time_comparision_table_data_real_dataset_single_algo_md(algo_name, exp_type, dataset_key)

            save_dir = os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '1-time_comparision_tables', 'per_dataset')
            os.makedirs(save_dir, exist_ok=True)
            with open(os.path.join(save_dir, f'{algo_name}_{exp_type}_{dataset_key}.md'), 'w') as f:
                f.write(table_str)

## 2. Line plots

In [8]:
import plotly.express as px
import os

for algo_name in ['LinGreedy', 'LinUCB', 'LinTS']:
    for use_log in [False, True]:
        for exp_type in exp_types:
            for num_items_num, num_items_str in num_items_dict.items():
                data = []
                for num_interactions_num, num_interactions_str in num_interactions_dict.items():
                    for full_algo_name in algorithms_list:
                        if algo_name in full_algo_name:
                            mean, std = get_experiment_mean_and_std(exp_type, full_algo_name, num_interactions_num, num_interactions_str, num_items_num, num_items_str)
                            data.append({
                                'Algorithm': full_algo_name.replace(f'_{algo_name}', '').replace('_', ' '),
                                'Interactions number': num_interactions_num,
                                'Mean execution time (s)': mean,
                                'Standard deviation (s)': std
                            })
                    
                df = pd.DataFrame(data)
                fig = px.line(df, x='Interactions number', y='Mean execution time (s)', color='Algorithm',
                                error_y='Standard deviation (s)', log_y=use_log)
                fig.update_layout(
                    legend_title_text='',
                    legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=-0.5,
                        xanchor="center",
                        entrywidth=77,
                        x=0.5
                    ),
                    margin=dict(b=10, t=0, r=0, l=0)
                )
                save_path = os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '2-line_plots', exp_type, algo_name)
                os.makedirs(save_path, exist_ok=True)
                fig.write_image(os.path.join(save_path, f'{algo_name}_{num_items_str}.jpg' if not use_log else f'{algo_name}_{num_items_str}_log.jpg'), width=300, height=300)
                fig.write_image(os.path.join(save_path, f'{algo_name}_{num_items_str}.svg' if not use_log else f'{algo_name}_{num_items_str}_log.svg'), width=300, height=300)

FileNotFoundError: [Errno 2] No such file or directory: 'results/toy_dataset_1k_100k/mab2rec_LinGreedy/not_incremental.csv'

In [ ]:

def generate_latex_figure_line_plot_svg(exp_type: str, num_items_num: int, num_items_str: str, use_log: bool):
    return f'''
\\begin{{figure}}[htbp]
    \centering
    \\begin{{subfigure}}[b]{{0.33\\textwidth}}
        \includesvg[width=\\textwidth]{{plots_and_tables/2-line_plots/{exp_type}/LinGreedy/LinGreedy_{num_items_str}{'_log' if use_log else ''}}}
        \caption{{LinGreedy}}
    \end{{subfigure}}
    \hfill
    \\begin{{subfigure}}[b]{{0.33\\textwidth}}
        \includesvg[width=\\textwidth]{{plots_and_tables/2-line_plots/{exp_type}/LinUCB/LinUCB_{num_items_str}{'_log' if use_log else ''}}}
        \caption{{LinUCB}}
    \end{{subfigure}}
    \hfill
    \\begin{{subfigure}}[b]{{0.33\\textwidth}}
        \includesvg[width=\\textwidth]{{plots_and_tables/2-line_plots/{exp_type}/LinTS/LinTS_{num_items_str}{'_log' if use_log else ''}}}
        \caption{{LinTS}}
    \end{{subfigure}}
    \caption{{Execution time comparison on toy datasets {exp_type.replace('_', ' ')} experiments with {num_items_num:,} ({num_items_str}) items.}}
\end{{figure}}
    '''

def generate_latex_figure_line_plot_jpg(exp_type: str, num_items_num: int, num_items_str: str, use_log: bool):
    return f'''
\\begin{{figure}}[htbp]
    \centering
    \\begin{{subfigure}}[b]{{0.33\\textwidth}}
        \includegraphics[width=\\textwidth]{{plots_and_tables/2-line_plots/{exp_type}/LinGreedy/LinGreedy_{num_items_str}{'_log' if use_log else ''}}}
        \caption{{LinGreedy}}
    \end{{subfigure}}
    \hfill
    \\begin{{subfigure}}[b]{{0.33\\textwidth}}
        \includegraphics[width=\\textwidth]{{plots_and_tables/2-line_plots/{exp_type}/LinUCB/LinUCB_{num_items_str}{'_log' if use_log else ''}}}
        \caption{{LinUCB}}
    \end{{subfigure}}
    \hfill
    \\begin{{subfigure}}[b]{{0.33\\textwidth}}
        \includegraphics[width=\\textwidth]{{plots_and_tables/2-line_plots/{exp_type}/LinTS/LinTS_{num_items_str}{'_log' if use_log else ''}}}
        \caption{{LinTS}}
    \end{{subfigure}}
    \caption{{Execution time comparison on toy datasets {exp_type.replace('_', ' ')} experiments with {num_items_num:,} ({num_items_str}) items.}}
\end{{figure}}
    '''

for exp_type in exp_types:
    for num_items_num, num_items_str in num_items_dict.items():
        with open(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '2-line_plots', exp_type, f'{num_items_str}_jpg.tex'), 'w') as f:
            f.write(generate_latex_figure_line_plot_jpg(exp_type, num_items_num, num_items_str, False))
        with open(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '2-line_plots', exp_type, f'{num_items_str}_log_jpg.tex'), 'w') as f:
            f.write(generate_latex_figure_line_plot_jpg(exp_type, num_items_num, num_items_str, True))
        with open(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '2-line_plots', exp_type, f'{num_items_str}_svg.tex'), 'w') as f:
            f.write(generate_latex_figure_line_plot_svg(exp_type, num_items_num, num_items_str, False))
        with open(os.path.join(PLOTS_AND_TABLES_SAVE_PATH, '2-line_plots', exp_type, f'{num_items_str}_log_svg.tex'), 'w') as f:
            f.write(generate_latex_figure_line_plot_svg(exp_type, num_items_num, num_items_str, True))
